In [1]:
# Basic
import mlflow
import pandas as pd
import warnings

# Machine Learning
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

warnings.filterwarnings("ignore")


# **Remote Tracking Server**

- Run en console: mlflow server --host 127.0.0.1 --port 5000

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("MLflow Quickstart")

<Experiment: artifact_location='mlflow-artifacts:/184226665751323025', creation_time=1767103795688, experiment_id='184226665751323025', last_update_time=1767103795688, lifecycle_stage='active', name='MLflow Quickstart', tags={}>

Verify Your Connection

In [3]:
# Print connection information
print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Active Experiment: {mlflow.get_experiment_by_name('MLflow Quickstart')}")

MLflow Tracking URI: http://127.0.0.1:5000
Active Experiment: <Experiment: artifact_location='mlflow-artifacts:/184226665751323025', creation_time=1767103795688, experiment_id='184226665751323025', last_update_time=1767103795688, lifecycle_stage='active', name='MLflow Quickstart', tags={}>


In [4]:
# Test logging
with mlflow.start_run():
    mlflow.log_param("test_param", "test_value")
    print("✓ Successfully connected to MLflow!")

✓ Successfully connected to MLflow!
🏃 View run sassy-crab-729 at: http://127.0.0.1:5000/#/experiments/184226665751323025/runs/2bed13a2137d458ca95e1dc2e378c104
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/184226665751323025


# **Machine Learning**

## Load dataset

In [5]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [6]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

In [7]:
# Enable autologging for scikit-learn
mlflow.sklearn.autolog()

# Just train the model normally
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

model_info = mlflow.sklearn.log_model(sk_model=lr, name="iris_model")

2025/12/30 09:19:43 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '5dfca81c2e314c5db13b5ce2a0ecf879', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run traveling-toad-696 at: http://127.0.0.1:5000/#/experiments/184226665751323025/runs/5dfca81c2e314c5db13b5ce2a0ecf879
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/184226665751323025


2025/12/30 09:19:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [8]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
